# Using Catalyst and ParaView in Jupyter Notebook

This is the minimal Version, just shortly showing, what you need in any case. A complete example can be seen in [Catalyst_Example](Catalyst_Example.ipynb).

This is setting up all components visible in the grafik below. Therefore giving you a notebook, that can use ParaView commands and is connected to a small sample simulation. The visualisation will be done in the notebook and is visible in your webbrowser.

<img src="img/Communication_minimal.png" width="400">

## Table of contents

- Requirements 
- Start Catalyst enabled simulation
- Setup Render Window in Browser
- Establish Catalyst Connection
- Setup render pipeline using ParaView Python Commands

## Requirements
- The bash variable JUPYTER_LOG_DIR needs to be set to a folder, where logs can be written. It can be set, before starting jupyter by executing `export JUPYTER_LOG_DIR="/your/path"` before running `jupyter-lab`. The two following lines allows to check the value that was set when starting jupyter:

In [ ]:
from os import environ
print(environ.get('JUPYTER_LOG_DIR'))

- Additional files are needed. They can be obtained by cloning the git repo, or copying the files in the folder `001-Catalyst2JupyterLab-Tutorial`. Afterwards the path to the folder needs to be written into the file `${JUPYTER_LAB_DIR}/catalystTutorialPath.txt`. This can be done with the following commands:

```
git init repo j4j_notebooks
cd j4j_notebooks
git remote add origin https://gitlab.version.fz-juelich.de/jupyter4jsc/j4j_notebooks.git
git config core.sparseCheckout true
echo "002-Methods/003-Visualization/001-InSitu/001-Catalyst2JupyterLab-Tutorial/*" >> .git/info/sparse-checkout
git pull --depth=1 origin master
cd 002-Methods/003-Visualization/001-InSitu/001-Catalyst2JupyterLab-Tutorial/
pwd >${JUPYTER_LOG_DIR}/catalystTutorialPath.txt
```

In [ ]:
import glob
file = open(environ.get('JUPYTER_LOG_DIR') + "/catalystTutorialPath.txt","r")
catalystTutorialPath = file.readline().rstrip()
file.close()
if len(catalystTutorialPath) < 30:
    import sys
    print("Path is short: '" + catalystTutorialPath + "' is it the right path?", file=sys.stderr)
    
if len(glob.glob(catalystTutorialPath+ "/*")) < 4:
    import sys
    print("Not enough files in '" + catalystTutorialPath + "' is it the right path?", file=sys.stderr)

## Start Catalyst enabled simulation
Here a small sample simulation is started, that will run on the same node, and send random pressure data

In [ ]:
%%script bash --bg --proc catalyst_process
cd `cat ${JUPYTER_LOG_DIR}/catalystTutorialPath.txt`
export OMP_NUM_THREADS=1
pvpython CatalystEnabledSimulation/fedriver.py CatalystEnabledSimulation/cpscript.py > ${JUPYTER_LOG_DIR}/simulation.log 2>&1

## Setup Render Window in Browser
- loading needed python modules
- Get URL to access this jupyter Lab
- Using pvlink connect with the websocket provided by ParaView. Additional examples and info for pvlink can be found [here](https://gitlab.version.fz-juelich.de/jupyter4jsc/j4j_extras/pvlink/-/blob/master/examples/Examples.ipynb "pvlink Examples").
- displaying the render window in a box, to get control over the size of the render window

In [ ]:
from paraview import simple
from pvlink import RemoteRenderer
from ipywidgets import Box
from os import environ

Jupyter_URL = "jupyter-jsc.fz-juelich.de" + environ.get('JUPYTERHUB_SERVICE_PREFIX', '/')
renderer = RemoteRenderer(baseURL="jupyter-jsc.fz-juelich.de" + environ.get('JUPYTERHUB_SERVICE_PREFIX', '/'), useJupyterServerProxyHttps=True, disableExternalPort=True)

Box(children=[renderer], layout={"height": "800px"})

Setup a view, that will be used by ParaView, setting up parameters for smooth rendering in this setup.

In [ ]:
from pvlink.utility import SetRecommendedRenderSettings, ResetCamera

# Cerate a view
view = simple.FindViewOrCreate("test", "RenderView")
SetRecommendedRenderSettings(view)
renderer.update_render()

## Establish Catalyst Connection
Create the Object handeling the connection, before extracting data from the Simulation. Waiting is necessary, Because extracting the data does not work, unless the simulation is connected and one update has been performed since updating

In [ ]:
catalyst = simple.CatalystConnection()

In [ ]:
# open port for catalyst connection
catalyst.Start()
catalyst.AddUpdateFunction(renderer.update_render)
catalyst.BlockTillConnected();

In [ ]:
# wait till simulation connected
catalyst.BlockTillConnected()
# extract data from simulation
# supplying a source name, that can be used to find the ParaView source.
# In case of different named input, or multiple input ports, alows to choose the desired input, that should be extracted
extract = catalyst.Extract("extract")
# block till there is an update for the simulation data
catalyst.BlockTillNextUpdate()
# display simulation data
simple.SetActiveSource(extract)
extractDisplay = simple.Show(extract, view)

ResetCamera(view, renderer)

Connection has been established, and the simulation output is now visible above in the render window

## Setup render pipeline using ParaView Python Commands
To get the wished visualisation we need to setup the ParaView pipeline, to tell it what it is supposed to do with the data

In [ ]:
# set scalar coloring
simple.ColorBy(extractDisplay, ("CELLS", "pressure"))

# rescale color and/or opacity maps used to include current data range
extractDisplay.RescaleTransferFunctionToDataRange(True, False)

# show color bar/color legend
extractDisplay.SetScalarBarVisibility(view, True)

# get color transfer function/color map for 'pressure'
pressureLUT = simple.GetColorTransferFunction("pressure")
pressureLUT.RescaleTransferFunction(0.0, 1.0)

# get opacity transfer function/opacity map for 'pressure'
pressurePWF = simple.GetOpacityTransferFunction("pressure")
pressurePWF.RescaleTransferFunction(0.0, 1.0)

# change representation type, for example wireframe or volume rendering
#extractDisplay.SetRepresentationType("Wireframe")
extractDisplay.SetRepresentationType("Volume");